In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import statistics

### import GEO reference data

In [2]:
# import GEO reference data
geoSamples = pd.read_csv(r'/scratch1/qiushipe/data_reusability/refs/geo_samples.csv', low_memory = False)
geoSeries = pd.read_csv(r'/scratch1/qiushipe/data_reusability/refs/geo_series.csv', low_memory = False)
geoSeries.rename(columns = {'Accession':'Series'}, inplace = True)

geoReference = pd.merge(geoSamples, geoSeries, how = 'outer', on = 'Series')[['Series', 'Accession', 'Platform', 'Datasets']].drop_duplicates()

In [3]:
geoReference

,Series,Accession,Platform,Datasets
0,GSE506,GSM1,GPL4,NaN
1,GSE506,GSM2,GPL4,NaN
2,GSE462,GSM3,GPL5,NaN
3,GSE462,GSM4,GPL5,NaN
4,GSE462,GSM5,GPL5,NaN
...,...,...,...,...
5001100,GSE198242,NaN,NaN,NaN
5001101,GSE198243,NaN,NaN,NaN
5001102,GSE198357,NaN,NaN,NaN
5001103,GSE198632,NaN,NaN,NaN


### import SRA reference data

In [4]:
# import SRA reference data
sraReference = pd.read_csv(r'/scratch1/qiushipe/data_reusability/refs/sra_complete_runs.csv', error_bad_lines = False, low_memory=False, quoting=3)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
sraReference.columns = ['Run','ReleaseDate','LoadDate','spots','bases','spots_with_mates','avgLength','size_MB','AssemblyName','download_path','Experiment','LibraryName','LibraryStrategy','LibrarySelection','LibrarySource','LibraryLayout','InsertSize','InsertDev','Platform','Model','SRAStudy','BioProject','Study_Pubmed_id','ProjectID','Sample','BioSample','SampleType','TaxID','ScientificName','SampleName','g1k_pop_code','source','g1k_analysis_group','Subject_ID','Sex','Disease','Tumor','Affection_Status','Analyte_Type','Histological_Type','Body_Site','CenterName','Submission','dbgap_study_accession','Consent','RunHash','ReadHash']

In [6]:
sraReference.head()

,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,SRR608890,2012-10-24 17:40:38,2015-06-28 12:45:31,1346028,403808400,0,300,245,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060605,NaN,public,546505D5AB5EF054BAA8143A4C243C36,360E2DABF823CE08914DFCCE9E5D508F
1,SRR608902,2012-10-24 18:25:10,2015-06-28 12:53:12,40463250,8092650000,0,200,5820,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060609,NaN,public,77FB2E9FDD76701CC4923EB5A149D077,FB14C111240DBBA90C10E6E379B6E6D4
2,SRR608903,2012-10-24 18:15:38,2015-06-28 12:52:57,39173373,7834674600,0,200,5658,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060611,NaN,public,DEFC477AF8D2B3463935B13B8D9FB4A0,879137FA07B9EFC4545B2E77C0A8C7DA
3,SRR608904,2012-10-24 18:24:10,2015-06-28 12:51:43,43730827,8746165400,0,200,6281,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060612,NaN,public,FE56F97D8ACB787E91BF8527E0D7FD7B,58A784648A6875021A9990D030F9B8C2
4,SRR608905,2012-10-24 18:01:08,2015-06-28 12:47:10,34155861,6831172200,0,200,4310,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060613,NaN,public,CB3DCAAEBCEB229416447548C567340A,6A47B22BA1E2D70121B5FAA0A7B14F45


In [7]:
# for collecting desired factors step
sraAttributes = sraReference

In [8]:
sraReference

,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,SRR608890,2012-10-24 17:40:38,2015-06-28 12:45:31,1346028,403808400,0,300,245,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060605,NaN,public,546505D5AB5EF054BAA8143A4C243C36,360E2DABF823CE08914DFCCE9E5D508F
1,SRR608902,2012-10-24 18:25:10,2015-06-28 12:53:12,40463250,8092650000,0,200,5820,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060609,NaN,public,77FB2E9FDD76701CC4923EB5A149D077,FB14C111240DBBA90C10E6E379B6E6D4
2,SRR608903,2012-10-24 18:15:38,2015-06-28 12:52:57,39173373,7834674600,0,200,5658,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060611,NaN,public,DEFC477AF8D2B3463935B13B8D9FB4A0,879137FA07B9EFC4545B2E77C0A8C7DA
3,SRR608904,2012-10-24 18:24:10,2015-06-28 12:51:43,43730827,8746165400,0,200,6281,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060612,NaN,public,FE56F97D8ACB787E91BF8527E0D7FD7B,58A784648A6875021A9990D030F9B8C2
4,SRR608905,2012-10-24 18:01:08,2015-06-28 12:47:10,34155861,6831172200,0,200,4310,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060613,NaN,public,CB3DCAAEBCEB229416447548C567340A,6A47B22BA1E2D70121B5FAA0A7B14F45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16824254,SRR9421177,2020-01-01 00:35:18,2019-07-27 07:13:53,221,161682,0,731,6,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,JCVI,SRA905006,NaN,public,3D82284D3F0228989239202B5DD9C6C7,782A4E2D51705A9FBB656A2B75E9B77B
16824255,SRR9421178,2020-01-01 00:35:19,2019-07-27 07:13:54,157,113691,0,724,4,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,JCVI,SRA905007,NaN,public,2A0FB2D8B4B0288582B93EC1EA59347D,769D3A6CDB42490F33D941BD940D6A74
16824256,SRR9421179,2020-01-01 00:35:19,2019-07-27 07:13:59,188,103086,0,548,6,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,JCVI,SRA905008,NaN,public,26332272163045B6A7B794FBB63A491A,3A76564E8A68064115DA9FAE0948E2B8
16824257,SRR9421180,2020-01-01 00:35:19,2019-07-27 07:13:59,208,116682,0,560,5,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,JCVI,SRA905009,NaN,public,4A4D021614192BE5829097A18DEF86E0,BF0AD03ADA519D8F49DFD3B716093584


In [9]:
sraReference = sraReference[['SRAStudy', 'Run', 'Experiment', 'BioProject', 'Submission', 'Sample']].drop_duplicates()

In [10]:
sraReference.head()

,SRAStudy,Run,Experiment,BioProject,Submission,Sample
0,SRP193094,SRR608890,SRX201320,PRJNA533584,SRA060605,SRS372941
1,SRP193094,SRR608902,SRX201313,PRJNA533584,SRA060609,SRS372941
2,SRP193094,SRR608903,SRX201313,PRJNA533584,SRA060611,SRS372941
3,SRP193094,SRR608904,SRX201319,PRJNA533584,SRA060612,SRS372941
4,SRP193094,SRR608905,SRX201312,PRJNA533584,SRA060613,SRS372941


In [11]:
# import data scraped from PubMed
pmcData = pd.read_csv(r'/scratch1/qiushipe/data_reusability/data_tables/pmcAndAccs.csv', names = ['pmc_ID', 'accession']).drop_duplicates()

In [12]:
pmcData

,pmc_ID,accession
0,PMC8516403,PRJNA606575
2,PMC8872613,SRX2234711
3,PMC8872613,SRR4408346
5,PMC8872613,SRR4408347
7,PMC8872613,SRR4408413
...,...,...
863519,PMC6219076,SRP132165
863524,PMC7066330,GSE143744
863526,PMC7066330,GSE143743
863528,PMC5714895,GSE101692


### Merge GEO accessions with reference data, convert to Series

In [13]:
# match series
Series_Merge = pd.merge(pmcData, geoReference['Series'].drop_duplicates(), how = 'left', left_on = 'accession', right_on = 'Series')
pmcData = Series_Merge.rename(columns = {'Series': 'Series_result'})

In [14]:
# match each other GEO ID (sample, platform, and dataset)
for subject in ['Accession', 'Platform', 'Datasets']:
    pmcData = pd.merge(pmcData, geoReference[['Series', subject]].drop_duplicates(subset = subject), how = 'left', 
            left_on = 'accession', right_on = subject)
    label = subject + '_result'
    pmcData = pmcData.rename(columns = {'Series': label})

In [15]:
pmcData

,pmc_ID,accession,Series_result,Accession_result,Accession,Platform_result,Platform,Datasets_result,Datasets
0,PMC8516403,PRJNA606575,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PMC8872613,SRX2234711,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PMC8872613,SRR4408346,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PMC8872613,SRR4408347,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PMC8872613,SRR4408413,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
400524,PMC6219076,SRP132165,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400525,PMC7066330,GSE143744,GSE143744,NaN,NaN,NaN,NaN,NaN,NaN
400526,PMC7066330,GSE143743,GSE143743,NaN,NaN,NaN,NaN,NaN,NaN
400527,PMC5714895,GSE101692,GSE101692,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# combine all GEO series match columns into one aggregate GEO series column, clean up
pmcData['geoSeries'] = pmcData['Series_result'].fillna(pmcData['Accession_result']).fillna(pmcData['Platform_result']).fillna(pmcData['Datasets_result'])
pmcData = pmcData.drop(labels = ['Accession', 'Platform', 'Datasets', 
                                 'Series_result', 'Accession_result',
                                 'Platform_result', 'Datasets_result'], axis = 1)
pmcData_geoMerged = pmcData

In [17]:
pmcData_geoMerged

,pmc_ID,accession,geoSeries
0,PMC8516403,PRJNA606575,NaN
1,PMC8872613,SRX2234711,NaN
2,PMC8872613,SRR4408346,NaN
3,PMC8872613,SRR4408347,NaN
4,PMC8872613,SRR4408413,NaN
...,...,...,...
400524,PMC6219076,SRP132165,NaN
400525,PMC7066330,GSE143744,GSE143744
400526,PMC7066330,GSE143743,GSE143743
400527,PMC5714895,GSE101692,GSE101692


### merge SRA accessions with reference data, convert to Study

In [18]:
# match SRA Study IDs
Study_Merge = pd.merge(pmcData_geoMerged, sraReference['SRAStudy'].drop_duplicates(), how = 'left', left_on = 'accession', right_on = 'SRAStudy')
pmcData = Study_Merge.rename(columns = {'SRAStudy': 'Study_result'})

In [19]:
# match every other SRA IDs (Run, Experiment, BioProject, Submission, Sample)
for subject in ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample']:
    pmcData = pd.merge(pmcData, sraReference[['SRAStudy', subject]].drop_duplicates(subset = [subject]), how = 'left', left_on = 'accession', right_on = subject)
    label = subject + '_result'
    pmcData = pmcData.rename(columns = {'SRAStudy': label})

In [20]:
# combine all SRA Study matches into one aggregate column, clean up
pmcData['sraStudy'] = pmcData['Study_result'].fillna(pmcData['Run_result']).fillna(pmcData['Experiment_result']).fillna(pmcData['BioProject_result']).fillna(pmcData['Submission_result']).fillna(pmcData['Sample_result'])
pmcData = pmcData.drop(labels = ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample',
                                'Study_result', 'Run_result', 'Experiment_result', 'BioProject_result',
                                'Submission_result', 'Sample_result'], axis = 1)
pmcData_sraMerged = pmcData

In [21]:
pmcData_sraMerged

,pmc_ID,accession,geoSeries,sraStudy
0,PMC8516403,PRJNA606575,NaN,SRP249567
1,PMC8872613,SRX2234711,NaN,SRP091318
2,PMC8872613,SRR4408346,NaN,SRP091318
3,PMC8872613,SRR4408347,NaN,SRP091318
4,PMC8872613,SRR4408413,NaN,SRP091318
...,...,...,...,...
400524,PMC6219076,SRP132165,NaN,SRP132165
400525,PMC7066330,GSE143744,GSE143744,NaN
400526,PMC7066330,GSE143743,GSE143743,NaN
400527,PMC5714895,GSE101692,GSE101692,NaN


In [22]:
# combine GEO Series hits and SRA study hits into one converted accession column
pmcData['converted_accession'] = pmcData_sraMerged['geoSeries'].fillna(pmcData_sraMerged['sraStudy'])
pmcData = pmcData.drop(labels = ['geoSeries', 'sraStudy'], axis = 1)

In [23]:
pmcData

,pmc_ID,accession,converted_accession
0,PMC8516403,PRJNA606575,SRP249567
1,PMC8872613,SRX2234711,SRP091318
2,PMC8872613,SRR4408346,SRP091318
3,PMC8872613,SRR4408347,SRP091318
4,PMC8872613,SRR4408413,SRP091318
...,...,...,...
400524,PMC6219076,SRP132165,SRP132165
400525,PMC7066330,GSE143744,GSE143744
400526,PMC7066330,GSE143743,GSE143743
400527,PMC5714895,GSE101692,GSE101692


### data cleaning

In [24]:
# clean out garbage converted_accession entries, and rows that didn't map to a converted_accession
w = []
for a in pmcData.converted_accession:
    if type(a)==str:
        if a[0:3] != 'GSE' or a[1:3] != 'RP':
            w.append(a)
pmcData = pmcData[pmcData.converted_accession.isin(w)]
pmcData

,pmc_ID,accession,converted_accession
0,PMC8516403,PRJNA606575,SRP249567
1,PMC8872613,SRX2234711,SRP091318
2,PMC8872613,SRR4408346,SRP091318
3,PMC8872613,SRR4408347,SRP091318
4,PMC8872613,SRR4408413,SRP091318
...,...,...,...
400524,PMC6219076,SRP132165,SRP132165
400525,PMC7066330,GSE143744,GSE143744
400526,PMC7066330,GSE143743,GSE143743
400527,PMC5714895,GSE101692,GSE101692


In [25]:
# perform QC with gold standard from Penn group (Casey + Kurt)
gsPMC = pd.read_table(r'/scratch1/qiushipe/data_reusability/refs/pubmed_mappings.tsv')
gsPMC.columns = ["SRA_accession_code", "GEO_accession_code", "pm_ID", "pmc_ID"]
gsPMC

,SRA_accession_code,GEO_accession_code,pm_ID,pmc_ID
0,SRP111833,GSE101341,29535194.0,PMC5850328
1,ERP108370,NaN,NaN,NaN
2,SRP062170,GSE71840,27780967.0,NaN
3,ERP114122,NaN,NaN,NaN
4,SRP162020,GSE120109,30692590.0,PMC6349857
...,...,...,...,...
26321,GSE140448,E-GEOD-140448,31776254.0,PMC6925996
26322,GSE137235,E-GEOD-137235,32467224.0,PMC7328513
26323,GSE140426,E-GEOD-140426,NaN,NaN
26324,GSE134244,E-GEOD-134244,NaN,NaN


In [26]:
# QC step: de-duplicate datasets present in both SRA and GEO
gsPMC_acc = gsPMC[['SRA_accession_code', 'GEO_accession_code']].dropna().drop_duplicates()
gsPMC_acc

,SRA_accession_code,GEO_accession_code
0,SRP111833,GSE101341
2,SRP062170,GSE71840
4,SRP162020,GSE120109
5,SRP041755,GSE57401
6,SRP153370,GSE117074
...,...,...
26321,GSE140448,E-GEOD-140448
26322,GSE137235,E-GEOD-137235
26323,GSE140426,E-GEOD-140426
26324,GSE134244,E-GEOD-134244


In [27]:
ovAcc = pd.merge(pmcData, gsPMC_acc, how = 'left', left_on = 'converted_accession', right_on = 'SRA_accession_code')
ovAcc

,pmc_ID,accession,converted_accession,SRA_accession_code,GEO_accession_code
0,PMC8516403,PRJNA606575,SRP249567,NaN,NaN
1,PMC8872613,SRX2234711,SRP091318,NaN,NaN
2,PMC8872613,SRR4408346,SRP091318,NaN,NaN
3,PMC8872613,SRR4408347,SRP091318,NaN,NaN
4,PMC8872613,SRR4408413,SRP091318,NaN,NaN
...,...,...,...,...,...
353428,PMC6219076,SRP132165,SRP132165,NaN,NaN
353429,PMC7066330,GSE143744,GSE143744,NaN,NaN
353430,PMC7066330,GSE143743,GSE143743,NaN,NaN
353431,PMC5714895,GSE101692,GSE101692,NaN,NaN


In [28]:
# count the number of SRA datasets also present in GEO
numDupSRA = len(ovAcc['SRA_accession_code']) - ovAcc['SRA_accession_code'].isna().sum()
print('duplicated SRA datasets: ' + str(numDupSRA))

duplicated SRA datasets: 5965


In [29]:
# convert SRA ID of duplicated datasets to GEO ID
ovAccNA = ovAcc.loc[ovAcc['SRA_accession_code'].isna(), :]
ovAcc = ovAcc.loc[~ovAcc['SRA_accession_code'].isna(), :]
ovAcc['converted_accession'] = ovAcc['GEO_accession_code']
ovAccNA

/tmp/SLURM_10293930/ipykernel_4358/722098114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovAcc['converted_accession'] = ovAcc['GEO_accession_code']


,pmc_ID,accession,converted_accession,SRA_accession_code,GEO_accession_code
0,PMC8516403,PRJNA606575,SRP249567,NaN,NaN
1,PMC8872613,SRX2234711,SRP091318,NaN,NaN
2,PMC8872613,SRR4408346,SRP091318,NaN,NaN
3,PMC8872613,SRR4408347,SRP091318,NaN,NaN
4,PMC8872613,SRR4408413,SRP091318,NaN,NaN
...,...,...,...,...,...
353428,PMC6219076,SRP132165,SRP132165,NaN,NaN
353429,PMC7066330,GSE143744,GSE143744,NaN,NaN
353430,PMC7066330,GSE143743,GSE143743,NaN,NaN
353431,PMC5714895,GSE101692,GSE101692,NaN,NaN


In [30]:
pmcData = pd.concat([ovAcc, ovAccNA], axis = 0) 
pmcData = pmcData[['pmc_ID', 'accession', 'converted_accession']]
pmcData

,pmc_ID,accession,converted_accession
50,PMC6279764,SRP073810,GSE80651
117,PMC8633104,SRP093349,GSE89843
238,PMC8542793,SRP010905,GSE35724
239,PMC8542793,SRP029602,GSE50612
240,PMC8542793,SRP029985,GSE50934
...,...,...,...
353428,PMC6219076,SRP132165,SRP132165
353429,PMC7066330,GSE143744,GSE143744
353430,PMC7066330,GSE143743,GSE143743
353431,PMC5714895,GSE101692,GSE101692


In [31]:
# count GEO number and SRA number
GEO_num = 0
SRA_num = 0
for i in pmcData.converted_accession:
    if i[0:3] == 'GSE':
        GEO_num = GEO_num + 1
    else:
        SRA_num = SRA_num + 1
print('GEO number:', GEO_num)
print('SRA number:', SRA_num)

GEO number: 223704
SRA number: 129729


### Collect desired factors

In [32]:
# Convert SRA dates to a universal format
sraAttributes['ReleaseDate'] = sraAttributes['ReleaseDate'].str[0:10]

In [33]:
sraAttributes.head()

,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,SRR608890,2012-10-24,2015-06-28 12:45:31,1346028,403808400,0,300,245,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060605,NaN,public,546505D5AB5EF054BAA8143A4C243C36,360E2DABF823CE08914DFCCE9E5D508F
1,SRR608902,2012-10-24,2015-06-28 12:53:12,40463250,8092650000,0,200,5820,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060609,NaN,public,77FB2E9FDD76701CC4923EB5A149D077,FB14C111240DBBA90C10E6E379B6E6D4
2,SRR608903,2012-10-24,2015-06-28 12:52:57,39173373,7834674600,0,200,5658,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060611,NaN,public,DEFC477AF8D2B3463935B13B8D9FB4A0,879137FA07B9EFC4545B2E77C0A8C7DA
3,SRR608904,2012-10-24,2015-06-28 12:51:43,43730827,8746165400,0,200,6281,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060612,NaN,public,FE56F97D8ACB787E91BF8527E0D7FD7B,58A784648A6875021A9990D030F9B8C2
4,SRR608905,2012-10-24,2015-06-28 12:47:10,34155861,6831172200,0,200,4310,NaN,https://sra-downloadb.st-va.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,WUGSC,SRA060613,NaN,public,CB3DCAAEBCEB229416447548C567340A,6A47B22BA1E2D70121B5FAA0A7B14F45


In [34]:
# Define functions to convert GEO dates to a universal format
def strToMonth(m):
    if(m == 'Jan'):
        return '01'
    elif(m == 'Feb'):
        return '02'
    elif(m == 'Mar'):
        return '03'
    elif(m == 'Apr'):
        return '04'
    elif(m == 'May'):
        return '05'
    elif(m == 'Jun'):
        return '06'
    elif(m == 'Jul'):
        return '07'
    elif(m == 'Aug'):
        return '08'
    elif(m == 'Sep'):
        return '09'
    elif(m == 'Oct'):
        return '10'
    elif(m == 'Nov'):
        return '11'
    elif(m == 'Dec'):
        return '12'
    else:
        return(np.NaN)
    
def convGEODate(d):
    if(type(d) == str):
        mon = strToMonth(d[0:3])
        day = d[4:6]
        yr = d[8:12]
        return yr + '-' + mon + '-' + day
    else:
        return np.NaN

In [35]:
# import GEO attribute data and add Series column
geoPlatforms = pd.read_csv('/scratch1/qiushipe/data_reusability/refs/geo_platforms.csv')
geoPlatforms.rename(columns={'Accession':'Platform'}, inplace = True)
techByPlatform = geoPlatforms[['Platform', 'Technology']]

# allData contains metadata matched to GEO series, but lacks 'Technology' column
allData = pd.merge(geoSamples, geoSeries, how = 'outer')
geoAttributes = pd.merge(allData, techByPlatform, how = 'left', on = 'Platform')

In [36]:
geoAttributes.head()

,Accession,Title,Sample_Type,Taxonomy,Channels,Platform,Series,Supplementary_Types,Supplementary_Links,SRA_Accession,Contact,Release_Date,Series_Type,Sample_Count,Datasets,PubMed_ID,Technology
0,GSM1,Foreskin Fibroblasts,SAGE,Homo sapiens,1.0,GPL4,GSE506,NaN,NaN,NaN,Marc Kenzelmann,"Sep 28, 2000",NaN,NaN,NaN,NaN,SAGE NlaIII
1,GSM2,HCMV-infected foreskin fibroblasts,SAGE,Homo sapiens,1.0,GPL4,GSE506,NaN,NaN,NaN,Marc Kenzelmann,"Sep 28, 2000",NaN,NaN,NaN,NaN,SAGE NlaIII
2,GSM3,testis a,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,"Oct 18, 2000",NaN,NaN,NaN,NaN,spotted DNA/cDNA
3,GSM4,testis b,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,"Oct 18, 2000",NaN,NaN,NaN,NaN,spotted DNA/cDNA
4,GSM5,male a,RNA,Drosophila melanogaster,1.0,GPL5,GSE462,NaN,NaN,NaN,Brian Oliver,"Oct 18, 2000",NaN,NaN,NaN,NaN,spotted DNA/cDNA


In [37]:
# convert GEO dates to universal format
dates = []
for i in geoAttributes['Release_Date']:
    dates.append(convGEODate(i))
geoAttributes['Release_Date'] = dates

In [38]:
# Add a column tagging each accession as GEO or SRA

repoList = []

for i in pmcData['converted_accession']:
    if(type(i) == str):
        if('GSE' in i or 'GPL' in i):
            repoList.append('GEO')
        elif('SRP' in i or 'ERP' in i or 'DRP' in i):
            repoList.append('SRA')
        else:
            repoList.append(np.NaN)
    else:
        repoList.append(np.NaN)
        
pmcData['repository'] = repoList

In [39]:
# add column for paper publish date
pmc_dates = pd.read_csv('/scratch1/qiushipe/data_reusability/data_lists/preFilterDates.csv', names = ['pmc_ID', 'date']).drop_duplicates()

pmcData = pd.merge(pmcData, pmc_dates, how = 'left', on = 'pmc_ID')
pmcData = pmcData.rename(columns = {'date': 'pmc_date'})

In [42]:
# Get every factor we're interested in from our tables of GEO and SRA metadata...

# take a slice of the GEO and SRA attribute tables with only the info we want
slicedGEOAtt = geoAttributes[['Series', 'Release_Date', 'Technology', 'Taxonomy']]
slicedGEOAtt.columns = ['converted_accession', 'geoRelease', 'geoHardware', 'geoSpecies']
slicedGEOAtt = slicedGEOAtt.drop_duplicates(subset = ['converted_accession'])

slicedSRAAtt = sraAttributes[['SRAStudy', 'ReleaseDate', 'Model', 
                              'LibraryStrategy', 'ScientificName', 
                              'bases', 'avgLength', 'Consent']]
slicedSRAAtt.columns = ['converted_accession', 'sraRelease', 'sraHardware', 
                        'sraLibrary_strategy', 'sraSpecies', 
                        'sraBases', 'sraAvg_length', 'sraAccess']
slicedSRAAtt = slicedSRAAtt.drop_duplicates(subset = ['converted_accession'])

In [43]:
slicedSRAAtt

,converted_accession,sraRelease,sraHardware,sraLibrary_strategy,sraSpecies,sraBases,sraAvg_length,sraAccess
0,SRP193094,2012-10-24,Illumina MiSeq,WGS,Astyanax mexicanus,403808400,300,public
17,SRP000694,2013-04-25,Illumina Genome Analyzer II,WGS,Drosophila melanogaster,9492281430,221,public
19,SRP000941,2010-11-30,Illumina Genome Analyzer II,ChIP-Seq,Homo sapiens,460555128,36,public
130,SRP020237,2011-07-26,Illumina Genome Analyzer IIx,WGS,Homo sapiens,437381928,72,DS-CA-MDS
185,SRP072529,2012-06-21,PacBio RS,WGS,Escherichia virus Lambda,89211659,1091,public
...,...,...,...,...,...,...,...,...
16824254,SRP204426,2020-01-01,AB 310 Genetic Analyzer,AMPLICON,Influenza A virus (A/environment/Ohio/1007/200...,161682,731,public
16824255,SRP204427,2020-01-01,AB 310 Genetic Analyzer,AMPLICON,Influenza A virus (A/environment/Ohio/994/2005...,113691,724,public
16824256,SRP204428,2020-01-01,AB 310 Genetic Analyzer,AMPLICON,Influenza A virus (A/equine/Alaska/29759/1991(...,103086,548,public
16824257,SRP204429,2020-01-01,AB 310 Genetic Analyzer,AMPLICON,Influenza A virus (A/equine/Algiers/1/1972(H3N8)),116682,560,public


In [44]:
# special case for GEO: make an educated guess on library strategy based on hardware
# These guesses are based on manually checking GEO series IDs that corresponded to various types of hardware

gc = Counter(geoAttributes['Technology'])
ls_guesses = pd.DataFrame.from_dict(gc, orient='index').reset_index()
ls_guesses.columns = ['hardware', 'use_count']
ls_guesses = ls_guesses.drop(labels = ['use_count'], axis = 1)

ls = []

for i in ls_guesses['hardware']:
    if(i == 'high-throughput sequencing'):
        ls.append('RNA-Seq')
    elif(i == 'SAGE NlaIII' or i == 'spotted DNA/cDNA' or i == 'SAGE Sau3A' 
         or i == 'in situ oligonucleotide' or i == 'spotted oligonucleotide' 
         or i == 'antibody' or i == 'MPSS' or i == 'oligonucleotide beads' 
         or i == 'RT-PCR' or i == 'mixed spotted oligonucleotide/cDNA' 
         or i == 'spotted peptide or protein'):
        ls.append('Expression_Array')
    else:
        ls.append(np.NaN)
        
ls_guesses.loc[:,'geoLibrary_strategy'] = ls

In [48]:
# merge SRA attributes onto pmcData table
mergedSRA = pd.merge(pmcData, slicedSRAAtt, how = 'left', on = 'converted_accession')
mergedSRA = mergedSRA.drop_duplicates()

# merge GEO attributes onto table of pmcData + SRA Attributes
allFactors = pd.merge(mergedSRA, slicedGEOAtt, how = 'left', on = 'converted_accession')
allFactors = pd.merge(allFactors, ls_guesses, how = 'left', left_on = 'geoHardware', right_on = 'hardware')

allFactors = allFactors.dropna(subset = ['converted_accession'])

In [49]:
allFactors

,pmc_ID,accession,converted_accession,repository,pmc_date,sraRelease,sraHardware,sraLibrary_strategy,sraSpecies,sraBases,sraAvg_length,sraAccess,geoRelease,geoHardware,geoSpecies,hardware,geoLibrary_strategy
0,PMC6279764,SRP073810,GSE80651,GEO,2018-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-10-24,high-throughput sequencing,Homo sapiens,high-throughput sequencing,RNA-Seq
1,PMC8633104,SRP093349,GSE89843,GEO,2021-11-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-15,high-throughput sequencing,Homo sapiens,high-throughput sequencing,RNA-Seq
2,PMC8542793,SRP010905,GSE35724,GEO,2021-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-18,high-throughput sequencing,Mus musculus,high-throughput sequencing,RNA-Seq
3,PMC8542793,SRP029602,GSE50612,GEO,2021-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,high-throughput sequencing,Mus musculus,high-throughput sequencing,RNA-Seq
4,PMC8542793,SRP029985,GSE50934,GEO,2021-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-06-10,high-throughput sequencing,Mus musculus,high-throughput sequencing,RNA-Seq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353428,PMC6219076,SRP132165,SRP132165,SRA,2018-11-06,2018-03-29,Illumina HiSeq 2500,miRNA-Seq,Spodoptera frugiperda,1721085666,20,public,NaN,NaN,NaN,NaN,NaN
353429,PMC7066330,GSE143744,GSE143744,GEO,2020-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-17,high-throughput sequencing,Homo sapiens,high-throughput sequencing,RNA-Seq
353430,PMC7066330,GSE143743,GSE143743,GEO,2020-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-17,high-throughput sequencing,Homo sapiens,high-throughput sequencing,RNA-Seq
353431,PMC5714895,GSE101692,GSE101692,GEO,2017-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-12,high-throughput sequencing,Glycine max,high-throughput sequencing,RNA-Seq


In [50]:
# clean up columns with factor for both SRA and GEO, rearrange columns
allFactors['species'] = allFactors['sraSpecies'].fillna(allFactors['geoSpecies'])
allFactors = allFactors.drop(labels = ['sraSpecies', 'geoSpecies'], axis = 1)

allFactors['hardware'] = allFactors['sraHardware'].fillna(allFactors['geoHardware'])
allFactors = allFactors.drop(labels = ['sraHardware', 'geoHardware'], axis = 1)

allFactors['library_strategy'] = allFactors['sraLibrary_strategy'].fillna(allFactors['geoLibrary_strategy'])
allFactors = allFactors.drop(labels = ['sraLibrary_strategy', 'geoLibrary_strategy'], axis = 1)

allFactors['repository_date'] = allFactors['sraRelease'].fillna(allFactors['geoRelease'])
allFactors = allFactors.drop(labels = ['sraRelease', 'geoRelease'], axis = 1)

cols = ['pmc_ID', 'accession', 'converted_accession', 'repository', 
        'pmc_date', 'repository_date', 'species', 
        'hardware', 'library_strategy', 'sraAvg_length', 'sraBases', 'sraAccess']

allFactors = allFactors[cols]
allFactors

,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,sraAccess
0,PMC6279764,SRP073810,GSE80651,GEO,2018-12-04,2016-10-24,Homo sapiens,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
1,PMC8633104,SRP093349,GSE89843,GEO,2021-11-12,2017-08-15,Homo sapiens,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
2,PMC8542793,SRP010905,GSE35724,GEO,2021-10-11,2013-10-18,Mus musculus,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
3,PMC8542793,SRP029602,GSE50612,GEO,2021-10-11,2015-01-01,Mus musculus,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
4,PMC8542793,SRP029985,GSE50934,GEO,2021-10-11,2014-06-10,Mus musculus,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
353428,PMC6219076,SRP132165,SRP132165,SRA,2018-11-06,2018-03-29,Spodoptera frugiperda,Illumina HiSeq 2500,miRNA-Seq,20,1721085666,public
353429,PMC7066330,GSE143744,GSE143744,GEO,2020-02-20,2020-01-17,Homo sapiens,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
353430,PMC7066330,GSE143743,GSE143743,GEO,2020-02-20,2020-01-17,Homo sapiens,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
353431,PMC5714895,GSE101692,GSE101692,GEO,2017-11-30,2018-01-12,Glycine max,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN


In [51]:
# save to .csv
allFactors.to_csv(r'/scratch1/qiushipe/data_reusability/data_tables/Metadata_Matrix_raw.csv', index = False)